In [47]:
from keras.layers import RepeatVector,Dense,Embedding,LSTM,TimeDistributed,Concatenate,Activation
from keras.models import Sequential,Model
from keras.optimizers import RMSprop

In [45]:
embedding_size=128
max_len=39
vocab_size=7669

In [53]:
def create_model(embedding_size,max_len):
    image_model=Sequential()
    image_model.add(Dense(embedding_size,input_shape=(2048,),activation="relu"))
    image_model.add(RepeatVector(max_len))
    #yield  image_model.summary()
    
    language_model=Sequential()
    language_model.add(Embedding(input_dim=vocab_size,output_dim=embedding_size,input_length=max_len))
    language_model.add(LSTM(256,return_sequences=True))
    language_model.add(TimeDistributed(Dense(embedding_size)))
    #yield language_model.summary()
    
    
    conc=Concatenate()([image_model.output,language_model.output])
    x=LSTM(128,return_sequences=True)(conc)
    x=LSTM(512,return_sequences=False)(x)
    x=Dense(vocab_size)(x)
    out=Activation("softmax")(x)
    model=Model(inputs=[image_model.input,language_model.input],outputs=out)
    model.compile(loss="categorical_crossentropy",optimizer="RMSprop",metrics=["accuracy"])
    print("model created!")
    return model.summary()


In [54]:
model=create_model(embedding_size,39)
model

model created!
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_4_input (InputLayer)  (None, 39)           0                                            
__________________________________________________________________________________________________
dense_12_input (InputLayer)     (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 39, 128)      981632      embedding_4_input[0][0]          
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 128)          262272      dense_12_input[0][0]             
______________________________________________________________________________________________

model created!
